# Lecture 26 – Visualizing Two Numerical Variables

## Data 6, Summer 2021

In [ ]:
from datascience import *
import numpy as np
Table.interactive_plots()

Our first dataset today comes from [Basketball Reference](https://www.basketball-reference.com/leagues/NBA_2020_per_game.html). It contains per-game averages of players in the 2019-2020 NBA season.

Run the cell below to load it in, select the relevant columns, and do some data cleaning.

**Note:** Most of the interesting data comes from the "better" players in the league; we will only look at players who averaged at least 10 points per game in the season. This isn't perfect, since there were plenty of good players who averaged less than 10 points per game.

In [ ]:
nba = Table.read_table('data/nba-2020.csv') \
           .select('Player', 'Pos', 'Tm', 'PTS', 'TRB', 'AST', '3PA', '3P%') \
           .where('3PA', are.not_equal_to(0))

def remove_code(name):
    return name[:name.index('\\')]

def get_court(pos):
    if 'G' in pos:
        return 'Guard'
    else:
        return 'Forward'

nba = nba.with_columns('Player', nba.apply(remove_code, 'Player'),
                       'Pos', nba.apply(get_court, 'Pos')) \
         .where('PTS', are.above(10))

In [ ]:
nba

A description of each column:

- `'Player'`: name
- `'Pos'`: general position (either Forward or Guard)
- `'Tm'`: abbreviated team
- `'PTS'`: average number of points scored per game
- `'TRB'`: average number of rebounds per game (a player receives a rebound when they grab the ball after someone misses)
- `'AST'`: average number of assists per game (a player receives an assist when they pass the ball to someone who then scores)
- `'3PA'`: average number of three-point shots attempted per game (a three point shot is one from behind a certain line, which is between 22-24 feet from the basket)
- `'3P%'`: average proportion of three-point shots that go in

## Review – bar charts and histograms

### Bar charts

In [ ]:
nba.group('Pos', np.mean).select('Pos', 'PTS mean', 'TRB mean', 'AST mean')

In [ ]:
nba.group('Pos', np.mean).select('Pos', 'PTS mean', 'TRB mean', 'AST mean').barh('Pos')

### Histograms

In [ ]:
nba.hist('PTS', density = False, bins = np.arange(10, 40, 2.5),
         width = 400, height = 600)

In [ ]:
nba.hist('TRB', density = False, group = 'Pos', bins = np.arange(17),
         xaxis_title = 'Rebounds',
         title = 'Distribution of Rebounds')

## Scatter plots

In [ ]:
example_data = Table().with_columns(
    'x', np.array([1, 4, 4, 3, 6]),
    'y', np.array([-1, 2, 8, 0, 1])
)

example_data

In [ ]:
example_data.scatter('x', 'y', s = 50, width = 500, height = 500)

In [ ]:
nba

### Example 1

In [ ]:
nba.scatter('PTS', 'AST')

Observation: On average, as the number of points a player averages increases, the number of assists they average also increases.

### Example 2

In [ ]:
nba.where('Pos', 'Forward') \
   .scatter('TRB', '3PA',
           xaxis_title = 'Rebounds Per Game (TRB)',
           yaxis_title = 'Three-Point Attempts Per Game (3PA)',
           title = '3PA vs. TRB for Forwards',
           width = 800,
           height = 500)

Observation: on average, as the number of rebounds a player averages per game increases, the number of three point attempts they average per game decreases.

### Quick Check 1

In [ ]:
# nba.where(..., ...) \
#    .scatter(..., ...,
#             xaxis_title = 'Points Per Game (PTS)', yaxis_title = 'Three-Point Percentage (3P%)',
#             title = '3P% vs. PTS for Players with at least 2 3PA',
#             width = 700, height = 500)

Observation: on average, as the number of points per game a player averages increases, three-point percentage neither increases nor decreases. (In other terms – it appears that PTS and 3P% are **uncorrelated**.)

## More customization

### Point size

In [ ]:
nba.scatter('PTS', '3P%', s = 40)

In [ ]:
nba.scatter('PTS', '3P%', s = 40, sizes = '3PA')

### Point color by grouping

In [ ]:
nba.scatter('TRB', '3PA', group = 'Pos', s = 30)

Observation: Guards tend to have fewer rebounds and more three-point attempts than forwards, who tend to have more rebounds and fewer three-point attempts.

### Labels

In [ ]:
nba.where('PTS', are.above(25)) \
   .scatter('PTS', 'AST',
           labels = 'Player',
           s = 30,
           width = 500,
           height = 500)

In [ ]:
nba.where('PTS', are.above(20)) \
   .scatter('PTS', 'AST',
           labels = 'Player',
           s = 30,
           sizes = '3PA',
           title = 'Players Averaging at least 20 PTS')

## Line plots

In [ ]:
nba_yearly = Table.read_table('data/nba-league-averages.csv') \
                  .select('Season', 'PTS', 'FGA', '3PA', '3P%', 'Pace')
nba_yearly = nba_yearly.with_columns('Season', np.arange(2021, 1979, -1))
nba_yearly

Our second dataset also comes from Basketball Reference. This dataset contains team-based average statistics for each year.

A little bit about our new dataset:
- `'Season'`: the second calendar year for each season (e.g. `2018` refers to the 2017-18 season)
- `'FGA'`: the average number of field goal attempts (shot attempts) per game
- `'Pace'`: the average number of times a team had possession of the ball per game

### Example 1

In [ ]:
nba_yearly.plot('Season', 'Pace')

Observation: The league slowed down in the late 90s and early 2000s, but is speeding back up.

### Example 2

In [ ]:
nba_yearly.plot('Season', '3PA',
               yaxis_title = 'Three-Point Attempts (3PA)',
               title = 'Three-Point Attempts Per Season',
               width = 700)

Observation: The three-point shot has rapidly increased in popularity over the past decade.

### Example 3

In [ ]:
nba_yearly.select('Season', 'FGA', '3PA')

In [ ]:
# Notice how we only supplied `plot` with a single argument
nba_yearly.select('Season', 'FGA', '3PA').plot('Season')

### Quick Check 2

In [ ]:
champ = nba_yearly.take(np.arange(1, 7)).select('Season', 'PTS', '3PA', 'Pace').with_columns(
    'Champion', np.array(['LAL', 'TOR', 'GSW', 'GSW', 'CLE', 'GSW'])
).select(0, -1, 1, 2, 3)

champ

In [ ]:
# champ.select('Season', 'Pace').barh('Season')

In [ ]:
# champ.scatter('PTS', 'Pace', s = 100, labels = 'Champion')

In [ ]:
# champ.scatter('PTS', 'Pace', s = 100, labels = 'Season')

In [ ]:
# champ.select('Season', 'PTS', 'Pace').plot('Season')